In [1]:
# read in our usual things
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets

In [2]:
# reading in a file reader for our simulation data
from hermite_library import read_hermite_solution_from_file

In [3]:
# use this data reader to read in Kepler 11 data
planet_file = 'data/Kepler-11-savedSim.txt'

# reading in time, energy, postion (3d), velocity (3d)
t_h, E_h, r_h, v_h = read_hermite_solution_from_file(planet_file)

In [4]:
r_h.shape
# number of bodies (star + planets), in 3D space, 8800 time steps

(7, 3, 8800)

In [5]:
# read in planet dataset too
planets = pd.read_csv('https://raw.githubusercontent.com/jnaiman/csci-p-14110_su2020/master/lesson08/planets_2020.06.22_10.10.17.csv', 
                      comment="#")

In [6]:
planets.head()

,rowid,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_controvflag,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,...,st_bmy,st_bmyerr,st_bmylim,st_m1,st_m1err,st_m1lim,st_c1,st_c1err,st_c1lim,st_colorn
0,1,11 Com,b,11 Com b,Radial Velocity,0,1,326.03000,0.32,-0.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
1,2,11 UMi,b,11 UMi b,Radial Velocity,0,1,516.21997,3.20,-3.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,3,14 And,b,14 And b,Radial Velocity,0,1,185.84000,0.23,-0.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
3,4,14 Her,b,14 Her b,Radial Velocity,0,1,1773.40002,2.50,-2.50,...,0.537,0.001,0.0,0.366,0.002,0.0,0.438,0.006,0.0,9.0
4,5,16 Cyg B,b,16 Cyg B b,Radial Velocity,0,1,798.50000,1.00,-1.00,...,0.418,0.003,0.0,0.222,0.003,0.0,0.351,0.003,0.0,17.0


We are going to plot (in 3D) our simulated data, but we want to make aesthetical choices about how the planets/star will look based on the observed data.

Select (based on a boolean mask) just the Kepler-11 data.

In [7]:
planets['pl_hostname']

0         11 Com
1         11 UMi
2         14 And
3         14 Her
4       16 Cyg B
          ...   
4159     tau Gem
4160     ups And
4161     ups And
4162     ups And
4163      xi Aql
Name: pl_hostname, Length: 4164, dtype: object

In [8]:
# subset the planets dataset based on having the hostname be Kepler-11
planets.loc[planets['pl_hostname'] == 'Kepler-11']

,rowid,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_controvflag,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,...,st_bmy,st_bmyerr,st_bmylim,st_m1,st_m1err,st_m1lim,st_c1,st_c1err,st_c1lim,st_colorn
1546,1547,Kepler-11,b,Kepler-11 b,Transit,0,6,10.3039,0.0006,-0.0010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1547,1548,Kepler-11,c,Kepler-11 c,Transit,0,6,13.0241,0.0013,-0.0008,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1548,1549,Kepler-11,d,Kepler-11 d,Transit,0,6,22.6845,0.0009,-0.0009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1549,1550,Kepler-11,e,Kepler-11 e,Transit,0,6,31.9996,0.0008,-0.0012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1550,1551,Kepler-11,f,Kepler-11 f,Transit,0,6,46.6888,0.0027,-0.0032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
1551,1552,Kepler-11,g,Kepler-11 g,Transit,0,6,118.3807,0.0010,-0.0006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0


In [9]:
# only interested in a subset of columns -- planet name, letter, mass, etc
planets.loc[planets['pl_hostname'] == 'Kepler-11', 
            ['pl_hostname', 'pl_letter', 'pl_name', 'pl_bmassj', 'pl_radj']]

,pl_hostname,pl_letter,pl_name,pl_bmassj,pl_radj
1546,Kepler-11,b,Kepler-11 b,0.006,0.161
1547,Kepler-11,c,Kepler-11 c,0.009,0.256
1548,Kepler-11,d,Kepler-11 d,0.023,0.278
1549,Kepler-11,e,Kepler-11 e,0.025,0.374
1550,Kepler-11,f,Kepler-11 f,0.006,0.222
1551,Kepler-11,g,Kepler-11 g,0.079,0.297


### Setting up our directory/file structure

So Python will know where to look for things

In [10]:
# 1. set the path to where the generic planet files are
generic_dir = './genericPlanetFiles/'

In [11]:
# 2. tell Python where to look for a .py file
#.  Appending Pythons "search" path
from sys import path

In [12]:
path # this is where Python is looking for library files with a .py at the end of them

['/Users/jillnaiman/csci-p-14110_su2020/lesson10',
 '/opt/anaconda3/envs/myNewEnv2/lib/python37.zip',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7/lib-dynload',
 '',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7/site-packages',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7/site-packages/IPython/extensions',
 '/Users/jillnaiman/.ipython']

In [13]:
path.append(generic_dir) # adding to this search file path

In [14]:
path # we can see genericPlanetFiles directory is now in this search list

['/Users/jillnaiman/csci-p-14110_su2020/lesson10',
 '/opt/anaconda3/envs/myNewEnv2/lib/python37.zip',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7/lib-dynload',
 '',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7/site-packages',
 '/opt/anaconda3/envs/myNewEnv2/lib/python3.7/site-packages/IPython/extensions',
 '/Users/jillnaiman/.ipython',
 './genericPlanetFiles/']

In [15]:
from make3dplanets import make3dplanets

In [16]:
# grabbing the radii information for Kepler-11
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']]
# .loc[row, column]

,pl_radj
1546,0.161
1547,0.256
1548,0.278
1549,0.374
1550,0.222
1551,0.297


In [17]:
# start formatting by grabbing just the values (w/o the fancy Pandas formatting)
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']].values

array([[0.161],
       [0.256],
       [0.278],
       [0.374],
       [0.222],
       [0.297]])

In [18]:
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']].values.shape

(6, 1)

In [19]:
# continuing formatting and making this "list" of numbers into just one array
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']].values.ravel() # .flatten()

array([0.161, 0.256, 0.278, 0.374, 0.222, 0.297])

In [20]:
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']].values.ravel().shape

(6,)

In [21]:
# save this formatted data into a radii vector
radii = planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']].values.ravel()

In [22]:
radii # jupiter radii

array([0.161, 0.256, 0.278, 0.374, 0.222, 0.297])

Based on prior knowledge we know that the star's radii (its 3D information) has to come last.

We have to add in the star's radius information as well.

In [23]:
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['st_rad']] # solar radii

,st_rad
1546,1.06
1547,1.06
1548,1.06
1549,1.06
1550,1.06
1551,1.06


In [24]:
# take off fancy Pandas formatting
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['st_rad']].values

array([[1.06],
       [1.06],
       [1.06],
       [1.06],
       [1.06],
       [1.06]])

In [25]:
# only need the one radius measurement
planets.loc[planets['pl_hostname'] == 'Kepler-11', ['st_rad']].values[0]

array([1.06])

In [26]:
star_radius = planets.loc[planets['pl_hostname'] == 'Kepler-11', ['st_rad']].values[0]

In [27]:
radii

array([0.161, 0.256, 0.278, 0.374, 0.222, 0.297])

In [28]:
# append this star_radius to our general radii array
# append using a NumPy function because these are NumPy arrays
radii = np.append(radii, star_radius)

In [29]:
radii
# all but the last radii are in Jupiter radii
# last radius is in solar radii

array([0.161, 0.256, 0.278, 0.374, 0.222, 0.297, 1.06 ])

We have to define a folder where we are going to store our 3D planet+host star files.

In [30]:
out_planet_dir = './outPlanets/'

In [31]:
SystemName = 'kepler11_take1' # unique file where this particular planet system will be stored
#systemName

We'll start with just one time step (and go from there).

In [32]:
Nplot = 0 # the first time step -- initial conditions for our data

In [33]:
# put everything together to make our first system
fname = make3dplanets(SystemName, r_h, radii, out_planet_dir, generic_dir, Nplot=0)

Going to overwrite files in  ./outPlanets/kepler11_take1/
Assuming radii are in Jupiter units except for the last particle which should have solar units, distances in AU... if not things are going to look weird!
no colors/textures specified -- generating random colors!


Because we can't see our system we're going to "fudge" our radii:

In [34]:
radii = planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']].values.ravel()*100
star_radius = planets.loc[planets['pl_hostname'] == 'Kepler-11', ['st_rad']].values[0]
radii = np.append(radii, star_radius)
radii

array([16.1 , 25.6 , 27.8 , 37.4 , 22.2 , 29.7 ,  1.06])

In [35]:
# redo making these planets
SystemName = 'kepler11_take2' # unique file where this particular planet system will be stored

In [36]:
fname = make3dplanets(SystemName, r_h, radii, out_planet_dir, generic_dir, Nplot=0)

Going to overwrite files in  ./outPlanets/kepler11_take2/
Assuming radii are in Jupiter units except for the last particle which should have solar units, distances in AU... if not things are going to look weird!
no colors/textures specified -- generating random colors!


In [53]:
# looking at WASP-47 system
planets.loc[planets['pl_hostname'] == 'WASP-47', 
            ['pl_hostname', 'pl_letter', 'pl_name', 'pl_bmassj', 'pl_radj']].dropna()

,pl_hostname,pl_letter,pl_name,pl_bmassj,pl_radj
4046,WASP-47,b,WASP-47 b,1.14243,1.127
4048,WASP-47,d,WASP-47 d,0.04122,0.319
4049,WASP-47,e,WASP-47 e,0.02149,0.161


In [54]:
SystemName = 'wasp47_take1'

In [55]:
radii = planets.loc[planets['pl_hostname'] == 'WASP-47', ['pl_radj']].dropna().values.ravel()*100
star_radius = planets.loc[planets['pl_hostname'] == 'WASP-47', ['st_rad']].dropna().values[0]
radii = np.append(radii, star_radius)
radii

array([112.7 ,  31.9 ,  16.1 ,   1.14])

In [56]:
fname = make3dplanets(SystemName, r_h, radii, out_planet_dir, generic_dir, Nplot=0)

Assuming radii are in Jupiter units except for the last particle which should have solar units, distances in AU... if not things are going to look weird!
no colors/textures specified -- generating random colors!


## Textures and texture mapping

Pick some textures for our planets.

In [77]:
planets.loc[planets['pl_hostname'] == 'Kepler-11', 
            ['pl_hostname', 'pl_letter', 'pl_name', 'pl_bmassj', 'pl_radj']].dropna()

,pl_hostname,pl_letter,pl_name,pl_bmassj,pl_radj
1546,Kepler-11,b,Kepler-11 b,0.006,0.161
1547,Kepler-11,c,Kepler-11 c,0.009,0.256
1548,Kepler-11,d,Kepler-11 d,0.023,0.278
1549,Kepler-11,e,Kepler-11 e,0.025,0.374
1550,Kepler-11,f,Kepler-11 f,0.006,0.222
1551,Kepler-11,g,Kepler-11 g,0.079,0.297


In [58]:
# Host star mass is 1.06 Solar masses -- just a little bigger than our sun

In theory we would want to look at the ratio of densities (mass/radius^3) -- this will tell us if we have gas planets or rocky planets.

We can make choices based on mass or radii -- as long as we are clear about what we are doing and why.

In [37]:
# approximate textures: Earth, Uranus, Uranus, Uranus, Earth, Saturn, Sun
planet_textures = ['earth_1200.jpg', 
                  'uranus_1024.jpg',
                  'uranus_1024.jpg',
                  'uranus_1024.jpg',
                   'earth_1200.jpg',
                   'saturn_1024.jpg',
                   'sun_texture1.jpg']

In [38]:
# overwrite take2 directory
SystemName = 'kepler11_take3' # unique file where this particular planet system will be stored

In [39]:
radii = planets.loc[planets['pl_hostname'] == 'Kepler-11', ['pl_radj']].values.ravel()*100
star_radius = planets.loc[planets['pl_hostname'] == 'Kepler-11', ['st_rad']].values[0]
radii = np.append(radii, star_radius)
radii

array([16.1 , 25.6 , 27.8 , 37.4 , 22.2 , 29.7 ,  1.06])

In [40]:
fname = make3dplanets(SystemName, r_h, radii, out_planet_dir, 
                      generic_dir, Nplot=0,
                     texture_file = planet_textures)

Assuming radii are in Jupiter units except for the last particle which should have solar units, distances in AU... if not things are going to look weird!


## Making 3D animations

Start with just 2 frames of our animation and use that to debug and then we can go from there.

In [63]:
r_h.shape

(7, 3, 8800)

We could play some games with downsampling, but we are going to just (right now) choose 2 particular time steps.

In [47]:
Nplots = [5,100] # 6th time step and the 101 timestep
#Nplots = np.arange(5,100,5)
Nplots

[5, 100]

In [48]:
SystemName = 'kepler11-movie'

In [49]:
dt_movie = 0.5 # seconds -- the time between showing different "frames" of my movie

In [50]:
# loop and create a bunch of obj files for each of our frames
# save the info we need for the sketchfab.timeframe file during this loop
# save the info to a file called sketchfab.timeframe

In [51]:
time_list = [] # store info for the sketcfab.timeframe file as a list

# fancy loop
for iplot, Nplot in enumerate(Nplots): # counting up the number of the file we have AND our selected timestep
    #print(iplot, Nplot)
    # this call here is creating the obj files
    fname = make3dplanets(SystemName, r_h, radii, out_planet_dir, 
                          generic_dir, Nplot=Nplot,
                         texture_file = planet_textures, 
                         fnum=iplot)
    # have to save information for the sketchfab.timeframe file
    time_list.append( str(dt_movie) + ' ' + fname + '\n')

Going to overwrite files in  ./outPlanets/kepler11-movie/
Assuming radii are in Jupiter units except for the last particle which should have solar units, distances in AU... if not things are going to look weird!
Assuming radii are in Jupiter units except for the last particle which should have solar units, distances in AU... if not things are going to look weird!


In [52]:
time_list

['0.5 kepler11-movie0000.obj\n', '0.5 kepler11-movie0001.obj\n']

In [53]:
# save the sketchfab.timeframe file
# make sure I'm saving to the correct location
timeframe_filename = out_planet_dir + SystemName + '/sketchfab.timeframe' # defining the name of the file

ftime = open(timeframe_filename,'w') # opening the file to write

for tobj in time_list: # looping over our saved information
    ftime.write(tobj) # printing to that file
    
# close file
ftime.close()

Trying to make a 10-step one:

In [59]:
#Nplots = [5,100] # 6th time step and the 101 timestep
Nplots = np.arange(0,100,10) # start, stop, step -- first 100 timesteps of the simulation in steps of 10
#Nplots = np.arange(0,8800,100) # start, stop, step - full sim, every 100 timesteps
len(Nplots)

10

In [60]:
Nplots

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [61]:
SystemName = 'kepler11-movie-10step'

In [62]:
time_list = [] # store info for the sketcfab.timeframe file as a list

# fancy loop
for iplot, Nplot in enumerate(Nplots): # counting up the number of the file we have AND our selected timestep
    #print(iplot, Nplot)
    # this call here is creating the obj files
    fname = make3dplanets(SystemName, r_h, radii, out_planet_dir, 
                          generic_dir, Nplot=Nplot,
                         texture_file = planet_textures, 
                         fnum=iplot, verbose=False)
    # have to save information for the sketchfab.timeframe file
    time_list.append( str(dt_movie) + ' ' + fname + '\n')

In [63]:
# make sure I'm saving to the correct location
timeframe_filename = out_planet_dir + SystemName + '/sketchfab.timeframe' # defining the name of the file

ftime = open(timeframe_filename,'w') # opening the file to write

for tobj in time_list: # looping over our saved information
    ftime.write(tobj) # printing to that file
    
# close file
ftime.close()

In [76]:
planets.loc[planets['pl_hostname'] == 'Kepler-107', 
            ['pl_hostname', 'pl_letter', 'pl_name', 'pl_bmassj', 'pl_radj']]
# Kepler 107

,pl_hostname,pl_letter,pl_name,pl_bmassj,pl_radj
1505,Kepler-107,b,Kepler-107 b,0.01104,0.137
1506,Kepler-107,c,Kepler-107 c,0.02954,0.142
1507,Kepler-107,d,Kepler-107 d,0.01196,0.077
1508,Kepler-107,e,Kepler-107 e,0.02706,0.259


In [65]:
!ls ../lesson09/data

55_Cnc-savedSim.txt     HD_215152-savedSim.txt  Kepler-411-savedSim.txt
DMPP-1-savedSim.txt     HD_219134-savedSim.txt  Kepler-62-savedSim.txt
GJ_3293-savedSim.txt    HD_34445-savedSim.txt   Kepler-79-savedSim.txt
GJ_667_C-savedSim.txt   HD_40307-savedSim.txt   Kepler-80-savedSim.txt
GJ_676_A-savedSim.txt   K2-266-savedSim.txt     Kepler-82-savedSim.txt
GJ_876-savedSim.txt     K2-285-savedSim.txt     TRAPPIST-1-savedSim.txt
HD_10180-savedSim.txt   KOI-94-savedSim.txt     WASP-47-savedSim.txt
HD_141399-savedSim.txt  Kepler-107-savedSim.txt myPlanetarySystem.txt
HD_160691-savedSim.txt  Kepler-11-savedSim.txt  tau_Cet-savedSim.txt
HD_20781-savedSim.txt   Kepler-20-savedSim.txt
HD_20794-savedSim.txt   Kepler-32-savedSim.txt
